Write a simple application to change the color balance of an image
by multiplying each color value by a different user-specified constant.

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [27]:
def adjust_color_balance(image_path, alpha=1.0, beta=1.0, gamma=1.0):
    image = cv2.imread(image_path)
    
    B, G, R = cv2.split(image)
    R = R * alpha
    G = G * beta
    B = B * gamma
    
    new_image = cv2.merge([B, G, R])
    # we can use both clipping and normalization. I prefer normalization however select clipping (reason explained later)
    # normalized_new_image = cv2.normalize(new_image, None, 0, 255, cv2.NORM_MINMAX) 
    normalized_new_image = np.clip(new_image, 0, 255)
    return normalized_new_image

image_path = 'image.png'
alpha, beta, gamma = 1.2, 1.0, 0.8
balanced_img = adjust_color_balance(image_path, alpha, beta, gamma)

original = cv2.imread(image_path)
cv2.imshow("original", original)
cv2.imshow("balanced_image", balanced_img.astype(np.uint8))

cv2.waitKey(0)
cv2.destroyAllWindows()

In [28]:
cv2.imwrite("color_balanced_image.png", balanced_img)

True

Do you get different results if you take out the gamma transformation before or after doing the multiplication? Why or why not?

Gamma correction is a non-linear transformation applied to images to adjust brightness and contrast in a way that aligns with human vision.

In [20]:
def apply_gamma(image, gamma):
    inv_gamma = 1.0 / gamma
    # Build a lookup table mapping the pixel values [0, 255] to their adjusted gamma values
    table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in np.arange(256)])
    transformed_image = cv2.LUT(image, table)
    return transformed_image


def adjust_color_balance_with_gamma(image_path, alpha=1.0, beta=1.0, gamma=1.0, apply_gamma_transformation_first=True, gamma_transformation_value=2.2):
    image = cv2.imread(image_path)
    
    if apply_gamma_transformation_first:
        image = apply_gamma(image, gamma_transformation_value)
    
    B, G, R = cv2.split(image)
    R = R * alpha
    G = G * beta
    B = B * gamma
    
    new_image = cv2.merge([B, G, R])
    # we can use both clipping and normalization. I prefer normalization
    normalized_new_image = cv2.normalize(new_image, None, 0, 255, cv2.NORM_MINMAX)
    # normalized_new_image = np.clip(new_image, 0, 255)
    
    if not apply_gamma_transformation_first:
        image = apply_gamma(image, gamma_transformation_value)
    
    return normalized_new_image


alpha, beta, gamma = 1.2, 1.0, 0.8
gamma_value = 2.2  # Common gamma value

# Gamma before scaling
balanced_img_gamma_first = adjust_color_balance_with_gamma(
    image_path, alpha, beta, gamma, True, gamma_value)

# Gamma after scaling
balanced_img_gamma_last = adjust_color_balance_with_gamma(
    image_path, alpha, beta, gamma, False, gamma_value)

cv2.imshow("original", original)
cv2.imshow("balanced_image_gamma_first", balanced_img_gamma_first.astype(np.uint8))
cv2.imshow("balanced_image_gamma_last", balanced_img_gamma_last.astype(np.uint8))

cv2.waitKey(0)
cv2.destroyAllWindows()

Hence we get different results by applying gamma transformation before and after color balancing. This is because gamma correction is a non-linear operation. The order in which these operations are applied affects how the scaling factors influence the final image's luminance and color balance.

Take the same picture with your digital camera using different color balance settings
(most cameras control the color balance from one of the menus). Can you recover what
the color balance ratios are between the different settings? You may need to put your
camera on a tripod and align the images manually or automatically to make this work.
Alternatively, use a color checker chart (Figure 10.3b), as discussed in Sections 2.3 and
10.1.1.

In [29]:
def compute_color_balance_ratios(image_path_A, image_path_B, num_samples=100):
    """
    Compute the color balance ratios between two images by sampling corresponding pixels.
    
    :param image_path_A: Path to image A (first setting)
    :param image_path_B: Path to image B (second setting)
    :param num_samples: Number of random samples to compute the ratios
    :return: (alpha, beta, gamma) scaling factors
    """
    # Load images
    image_A = cv2.imread(image_path_A)
    image_B = cv2.imread(image_path_B)
    
    # Ensure both images have the same dimensions
    if image_A.shape != image_B.shape:
        print("Error: Images do not have the same dimensions.")
        return
    
    # Flatten the images to a list of pixels channelwise
    pixels_A = image_A.reshape(-1, 3)
    pixels_B = image_B.reshape(-1, 3)
    
    # Randomly select sample pixels
    indices = np.random.choice(pixels_A.shape[0], num_samples, replace=False)
    sampled_A = pixels_A[indices]
    sampled_B = pixels_B[indices]
    
    # Compute ratios, avoiding division by zero
    ratios = sampled_B / (sampled_A + 1e-6)
    
    # Compute average ratios for each channel
    alpha = np.mean(ratios[:,2])  # Red channel
    beta = np.mean(ratios[:,1])   # Green channel
    gamma = np.mean(ratios[:,0])  # Blue channel
    
    return alpha, beta, gamma

# Example usage
image_path_A = 'image.png'
image_path_B = 'color_balanced_image.png'
alpha, beta, gamma = compute_color_balance_ratios(image_path_A, image_path_B, 1000)
print(alpha, beta, gamma)

1.19361249248255 0.9989999746495232 0.7958366635878162


The interesting thing I noticed is that you will not get accurate value if you had performed normalization, instead of clipping, while color balancing (normalization changes pixel value hence ratio is not maintained).

Can you think of any reason why you might want to perform a color twist (Sec-
tion 3.1.2) on the images? 

In [ ]:
def color_twist(image_path, hue_shift=0, saturation_scale=1.0, brightness_scale=1.0):
    """
    Perform a color twist on an image by adjusting hue, saturation, and brightness.
    
    :param image_path: Path to the input image
    :param hue_shift: Degrees to shift the hue (0-180 in OpenCV)
    :param saturation_scale: Scaling factor for saturation
    :param brightness_scale: Scaling factor for brightness/value
    :return: Twisted image in RGB format
    """
    # Load image
    image = cv2.imread(image_path).astype(np.float32)
    
    # Convert to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Adjust Hue
    hsv[:, :, 0] = (hsv[:, :, 0] + hue_shift) % 180  # Hue values in OpenCV are [0,179]
    
    # Adjust Saturation
    hsv[:, :, 1] = hsv[:, :, 1] * saturation_scale
    hsv[:, :, 1] = np.clip(hsv[:, :, 1], 0, 255)
    
    # Adjust Brightness/Value
    hsv[:, :, 2] = hsv[:, :, 2] * brightness_scale
    hsv[:, :, 2] = np.clip(hsv[:, :, 2], 0, 255)
    
    # Convert back to BGR
    twisted_image = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    
    return twisted_image

# Example usage
image_path = 'image.png'
hue_shift = 100  # Shift hue by 10 degrees
saturation_scale = 1.2  # Increase saturation by 20%
brightness_scale = 1.1  # Increase brightness by 10%

twisted_img = color_twist(image_path, hue_shift, saturation_scale, brightness_scale)

original = cv2.imread(image_path)

cv2.imshow("original", original)

# since cv2.imshow() displays image in uint8 format so we see artefacts
# save image to disk and view it for accurate results  
cv2.imshow("twisted image", twisted_img.astype(np.uint8))
# cv2.imwrite("twisted_image.png", twisted_img)

cv2.waitKey(0)
cv2.destroyAllWindows()

A Color Twist involves altering the hue, saturation, or brightness of an image to achieve desired visual effects or correct color imbalances. It's a form of color transformation that can enhance the aesthetic appeal or correct color issues in images.